# Install dependencies and download the data should take about 5 miutes

In [ ]:
!pip install datasets evaluate google-cloud-storage vaex transformers dataquality
import os
from tqdm import tqdm
from datasets import load_dataset
import torch 
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from matplotlib import pyplot as plt
from google.cloud import storage

# download images
import os
os.system('gsutil -m cp -r \
  "gs://galileo-public-data/CV_datasets/COCO_segmentation_example_data" \
  .')

dataset_path = "./COCO_segmentation_example_data"

IMG_SIZE = 512
NC = 21  # Number of classes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 114.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

#Create the dataset class


In [ ]:
from typing import Optional, Dict, Union
from PIL import Image

class coco_hf_dataset_disk(torch.utils.data.Dataset):
    def __init__(self, 
                 dataset_path: str,
                 relative_img_path: str, 
                 relative_mask_path: str,
                 mask_transform: transforms=None, 
                 img_transform: transforms=None, 
                 size: int=1024,) -> None:
        """"
        COCO val dataset from galileo-public-data/CV_datasets/COCO_seg_val_5000/all_images
        downloaded and located on disk.
        If no paths are provided we download the dataset from GCS and save it to disk.

        :param dataset_path: path to dataset
        :param relative_img_path: path to images relative to the dataset path
        :param relative_maks_path: path to masks relative to the dataset path
        :param mask_transform: transforms to apply to masks
        :param img_transform: transforms to apply to images
        :param size: size of image and mask
        """
        super(coco_hf_dataset_disk, self).__init__()

        self.dataset_path = dataset_path
        self.relative_img_path = relative_img_path
        self.relative_mask_path = relative_mask_path
        self.images = sorted(os.listdir(os.path.join(dataset_path, relative_img_path)))
        self.masks = sorted(os.listdir(os.path.join(dataset_path, relative_mask_path)))
        # remove .DS_Store
        if self.images[0] == '.DS_Store':
            self.images = self.images[1:]
        if self.masks[0] == '.DS_Store':
            self.masks = self.masks[1:]

        num_images = len(self.images)
        num_masks = len(self.masks)
        print(f"Found dataset, there are {num_images} images and {num_masks} masks")

        self.mask_transform = mask_transform
        self.img_transform = img_transform

        self.class_dict = { 'background': 0, 'airplane': 1, 'bicycle': 2,
                            'bird': 3, 'boat': 4, 'bottle': 5, 'bus': 6,
                            'car': 7, 'cat': 8, 'chair': 9, 'cow': 10,
                            'dining table': 11,'dog': 12,'horse': 13,
                            'motorcycle': 14,'person': 15,'potted plant': 16,
                            'sheep': 17, 'couch': 18, 'train': 19, 'tv': 20}
        self.labels = [i for i in self.class_dict]
                        
        self.int2str = {v: k for k, v in self.class_dict.items()}
        self.size = size

    def __len__(self) -> int:
        # returns length of the dataset
        return len(self.images)

    def __getitem__(self, idx: int) -> Dict[str, Union[torch.Tensor, int, np.ndarray]]:
        # gets a single item from our dataset
        
        image_path = os.path.join(self.dataset_path, self.relative_img_path, self.images[idx])
        mask_path = os.path.join(self.dataset_path, self.relative_mask_path, self.masks[idx])
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        # resize image and mask to given size
        unnormalized_image = image.copy().resize((self.size, self.size), resample=Image.NEAREST)
        unnormalized_image = transforms.ToTensor()(unnormalized_image)
        unnormalized_image = expand_gray_channel()(unnormalized_image)
        unnormalized_image = np.array(unnormalized_image)
        

        if self.img_transform:
            image = self.img_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        
        return {'image': image,
                'image_path': image_path,
                'mask_path': mask_path,
                'mask': mask,
                'idx': idx,
                'unnormalized_image': unnormalized_image}


class expand_gray_channel:
    def __call__(self, tensor: torch.Tensor) -> torch.Tensor:
        # torch transform to expand gray channel to 3 channels
        if tensor.shape[0] > 3:
            tensor = tensor.unsqueeze(0)
        if tensor.shape[0] == 1:
            return tensor.expand(3, -1, -1)
        return tensor
    

#Create the datasets, dataloaders, model and optimizer

In [ ]:
img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.BICUBIC),
    expand_gray_channel(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
mask_transforms = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.NEAREST),
])

train_image_path = 'train/images'
train_mask_path = 'train/masks'
validation_image_path = 'validation/images'
validation_mask_path = 'validation/masks'

train_dataset = coco_hf_dataset_disk(dataset_path=dataset_path,
                                    relative_img_path=train_image_path, 
                                    relative_mask_path=train_mask_path,
                                    img_transform=img_transforms,
                                     mask_transform=mask_transforms,
                                    size=IMG_SIZE)
validation_dataset = coco_hf_dataset_disk(dataset_path=dataset_path,
                                    relative_img_path=validation_image_path, 
                                    relative_mask_path=validation_mask_path,
                                    img_transform=img_transforms,
                                     mask_transform=mask_transforms,
                                    size=IMG_SIZE)
labels = train_dataset.labels
train_dataset = torch.utils.data.Subset(train_dataset, range(400))
validation_dataset = torch.utils.data.Subset(validation_dataset, range(400, 450))
train_dataloader = DataLoader(train_dataset, batch_size = 6, shuffle=True, num_workers=4)
validation_dataloader = DataLoader(validation_dataset, batch_size = 6, shuffle=True, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .00001)

# Dataquality

In [ ]:
import sys
sys.path.append(os.path.abspath("dataquality"))

In [ ]:
import dataquality as dq
from dataquality.integrations.cv.torch.semantic_segmentation import watch

dq.configure()
dq.init("semantic_segmentation", "Segmentation_Project", 'COCO_dataset')

watch(
    model,
    bucket_url='https://storage.googleapis.com/galileo-public-data/CV_datasets/COCO_segmentation_example_data',
    dataset_path=dataset_path,
    dataloaders={"training": train_dataloader,
                 "validation": validation_dataloader},
)
dq.set_labels_for_run(labels)


In [ ]:
# train for one epoch

scaler = torch.cuda.amp.GradScaler()
epochs = 1

with torch.autocast('cuda'):
    for epoch in range(epochs):
        dq.set_epoch_and_split(epoch, "training")
        for j, sample in enumerate(tqdm(train_dataloader)):
            imgs, masks = sample['image'], sample['mask']
            out = model(imgs.to(device))

            # reshape to have loss for each pixel (bs * h * w, 21)\n",
            pred = out['out'].permute(0, 2, 3, 1).contiguous().view( -1, 21)
            masks = masks.long()
            msks_for_loss = masks.view(-1).to(device)

            loss = criterion(pred, msks_for_loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [ ]:
import time
now = time.time()
dq.finish()
print(time.time() - now)